# Web Scrapping France Travail API Offres d'emploi

In [1]:
import requests
import os, sys
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import time

### Importation des variables d'environnements

In [2]:
load_dotenv()

BASE_URL = "https://api.francetravail.io/partenaire/offresdemploi"
TOKEN_URL = "https://entreprise.francetravail.fr/connexion/oauth2/access_token?realm=/partenaire"
PUBLIC_KEY = os.environ.get("FT_PUBLIC_KEY")
PRIVATE_KEY = os.environ.get("FT_PRIVATE_KEY")

### Fonction de récupération du token

In [3]:
def get_token():
    try:
        res = requests.post(TOKEN_URL, data={
            "grant_type": "client_credentials",
            "client_id":PUBLIC_KEY,
            "client_secret":PRIVATE_KEY,
            "scope": "api_offresdemploiv2 o2dsoffre"
        }
    )
        return res
    except Exception as err :
        sys.exit(err)

### Fonction de récupération des requêtes en fonction des paramètres

In [4]:
def get_response_from_url(url,params):
    try:
        token = get_token()
        if token.status_code == 200:
            token = token.json().get('access_token')
            
        res = requests.get(
            url,
            headers={"Authorization": f"Bearer {token}"},
            params=params
        )
        return res
    except Exception as err :
        sys.exit(err)

### Récupération du premier élément de l'ENDPOINTS "Rechercher des offres" dans l'API "Offres d'emploi"

In [5]:
res = get_response_from_url(f"{BASE_URL}/v2/offres/search",params={'range':"1-1"})
print(f"taille : {len(res.json()['resultats'])}")
res.json()['resultats']

taille : 1


[{'id': '190PKVQ',
  'intitule': 'Mécanicien / Mécanicienne agricole (H/F)',
  'description': "urgent!!!!\nmécanicien agricole polyvalent \nentretien et réparation tous matériels agricoles +VL +parc automobile\nsavoir être polyvalent et s'adapter aux demandes\npossibilité TRACTORISTE/ CHAUFFEUR\nSalaire négociable selon compétences\névolution possible selon profil\n\nposte NON logé",
  'dateCreation': '2025-04-03T14:15:51.738Z',
  'dateActualisation': '2025-04-03T14:15:52.276Z',
  'lieuTravail': {'libelle': '2B - LUCCIANA',
   'latitude': 42.542189,
   'longitude': 9.472991,
   'codePostal': '20290',
   'commune': '2B148'},
  'romeCode': 'I1603',
  'romeLibelle': 'Mécanicien-réparateur / Mécanicienne-réparatrice en matériels agricoles',
  'appellationlibelle': 'Mécanicien / Mécanicienne agricole',
  'entreprise': {'nom': 'SCEA SANTINI', 'entrepriseAdaptee': False},
  'typeContrat': 'CDD',
  'typeContratLibelle': 'CDD - 6 Mois',
  'natureContrat': 'Contrat travail',
  'experienceExige':

### Récupération de toutes les ranges possibles du paramètre range 

In [6]:
full_range_param = np.arange(0,3151,150)
full_range_param[-1] = full_range_param[-1]
full_range_param

array([   0,  150,  300,  450,  600,  750,  900, 1050, 1200, 1350, 1500,
       1650, 1800, 1950, 2100, 2250, 2400, 2550, 2700, 2850, 3000, 3150])

### Fonction de récupération de toutes les offres d'emploi

In [7]:
def get_all_data(range_call):
    res = np.array([])
    while len(range_call) > 1:
        d = range_call[0]
        p = range_call[1] - 1
        range_call = np.delete(range_call,0)

        print(f"{d}-{p}")
        res = np.append(
            res,
            get_response_from_url(f"{BASE_URL}/v2/offres/search",params={'range':'1-150'})
            .json()['resultats']
        )
        time.sleep(0.10)
        

    return res


In [8]:
res = get_all_data(full_range_param)

0-149
150-299
300-449
450-599
600-749
750-899
900-1049
1050-1199
1200-1349
1350-1499
1500-1649
1650-1799
1800-1949
1950-2099
2100-2249
2250-2399
2400-2549
2550-2699
2700-2849
2850-2999
3000-3149


In [9]:
df = pd.DataFrame(res.tolist())

In [10]:
df.shape

(3150, 41)

In [11]:
df.head()

,id,intitule,description,dateCreation,dateActualisation,lieuTravail,romeCode,romeLibelle,appellationlibelle,entreprise,...,secteurActivite,secteurActiviteLibelle,origineOffre,offresManqueCandidats,contexteTravail,qualitesProfessionnelles,formations,experienceCommentaire,complementExercice,conditionExercice
0,190PKVQ,Mécanicien / Mécanicienne agricole (H/F),urgent!!!!\nmécanicien agricole polyvalent \ne...,2025-04-03T14:15:51.738Z,2025-04-03T14:15:52.276Z,"{'libelle': '2B - LUCCIANA', 'latitude': 42.54...",I1603,Mécanicien-réparateur / Mécanicienne-réparatri...,Mécanicien / Mécanicienne agricole,"{'nom': 'SCEA SANTINI', 'entrepriseAdaptee': F...",...,01,"Culture de légumes, de melons, de racines et d...","{'origine': '1', 'urlOrigine': 'https://candid...",False,{'horaires': ['35H Travail en journée']},NaN,NaN,NaN,NaN,NaN
1,190PKVP,CDD AGENT D'EXPLOITATION DES ROUTES PLOËRMEL (...,Votre environnement de travail:\nLa Direction ...,2025-04-03T14:15:40.636Z,2025-04-03T14:15:41.303Z,"{'libelle': '56 - Ploërmel', 'latitude': 47.91...",I1202,Agent / Agente d'entretien de la voirie,Agent / Agente d'exploitation de la voirie,"{'nom': 'DEPARTEMENT DU MORBIHAN', 'entreprise...",...,84,Administration publique générale,"{'origine': '1', 'urlOrigine': 'https://candid...",False,{'horaires': ['39H Travail en journée']},"[{'libelle': 'Faire preuve d'autonomie', 'desc...",NaN,NaN,NaN,NaN
2,190PKVN,Shopping Planner (H/F),Devenez Shopping Planner Indépendant(e) chez E...,2025-04-03T14:15:39.510Z,2025-04-03T14:15:40.076Z,"{'libelle': '14 - CAEN', 'latitude': 49.184316...",D1403,Commercial / Commerciale auprès des particuliers,Conseiller vendeur / Conseillère vendeuse à do...,"{'nom': 'ELORA', 'description': 'Elora, la réf...",...,47,Vente à domicile,"{'origine': '1', 'urlOrigine': 'https://candid...",False,{},"[{'libelle': 'Faire preuve de persévérance', '...",NaN,NaN,NaN,NaN
3,190PKVM,Technicien / Technicienne d'entretien et de ma...,Notre agence de travail temporaire TRIANGLE IN...,2025-04-03T14:15:36.552Z,2025-04-03T14:15:37.214Z,"{'libelle': '50 - LA HAGUE', 'latitude': 49.65...",I1203,Agent / Agente d'entretien du bâtiment,Technicien(ne) d'entretien et de maintenance d...,"{'nom': 'TRIANGLE SOLUTIONS RH', 'description'...",...,78,Activités des agences de travail temporaire,"{'origine': '1', 'urlOrigine': 'https://candid...",False,{'horaires': ['37H Travail en journée']},[{'libelle': 'Organiser son travail selon les ...,NaN,NaN,NaN,NaN
4,190PKVL,Animateur / Animatrice périscolaire (H/F),Nous recherchons un-e Animateur-trice périscol...,2025-04-03T14:15:33.088Z,2025-04-03T14:15:33.678Z,"{'libelle': '67 - Ernolsheim-Bruche', 'latitud...",G1203,Animateur / Animatrice auprès des jeunes,Animateur / Animatrice d'activités périscolaires,{'nom': 'ASS DE LOISIRS EDUCATIFS ET DE FORMAT...,...,88,Autre accueil ou accompagnement sans hébergeme...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,{'horaires': ['temps partiel - 10H Travail en ...,"[{'libelle': 'Faire preuve d'autonomie', 'desc...","[{'codeFormation': '44054', 'domaineLibelle': ...",NaN,NaN,NaN


### Export csv

In [12]:
#df.to_csv('full_recherche_emploi_brut.csv')